In [38]:
import os
import urllib.request
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm

Until now we have: 
   - 208 Cataracts. 175 from previous set and 33 from latest set 
   - 322 Healthy. 200 from previous set and 122 from latest set

In [58]:
path = 'Datasets\\UNHAS'
filename='UnhasImagesCataractsCropped.csv'
filename2='UnhasImagesHealthyCropped.csv'
csv_path=os.path.join(path,filename)
csv_path2=os.path.join(path,filename2)
df_cataracts = pd.read_csv(csv_path)
df_healthy = pd.read_csv(csv_path2)

In [65]:
print(df_cataracts.head())
print(df_healthy.head())

       folder                                        filename  height  width  \
0  17-03-2021      WhatsApp Image 2021-03-17 at 08.28.40.jpeg    1280    988   
1  17-03-2021  WhatsApp Image 2021-03-17 at 08.16.08 (1).jpeg     984   1045   
2  17-03-2021      WhatsApp Image 2021-03-17 at 08.28.38.jpeg     227    324   
3  17-03-2021  WhatsApp Image 2021-03-17 at 08.16.12 (1).jpeg     270    300   
4  17-03-2021      WhatsApp Image 2021-03-24 at 11.53.24.jpeg     992   1280   

   bands  filesize(KB)  
0      3       161.712  
1      3       550.775  
2      3        42.335  
3      3        50.687  
4      3       130.944  
          folder                                        filename  height  \
0  Koass unismuh      WhatsApp Image 2021-03-20 at 08.33.13.jpeg     978   
1  Koass unismuh  WhatsApp Image 2021-03-23 at 12.10.21 (2).jpeg     914   
2  Koass unismuh      WhatsApp Image 2021-03-24 at 10.39.05.jpeg    1016   
3  Koass unismuh      WhatsApp Image 2021-03-24 at 15.34.55.jpeg 

In [46]:
#Reading cataract images
cat_images = []
noread = []
path_cat_images=os.path.join(path,'UnhasImagesCataractsCropped')
file_names = df_cataracts.filename
row=64
col=64

for image_name in tqdm(file_names):
    try:
        image = cv2.imread(os.path.join(path_cat_images,image_name))
        image = cv2.resize(image,(row,col))
        cat_images.append(image)
    except:
        noread.append(image_name)
        continue

cat_images = np.array(cat_images)
noread = np.array(noread)

100%|███████████████████████████████████████████████████████████████████████████████| 208/208 [00:00<00:00, 248.17it/s]


In [66]:
cat_images.shape

(208, 64, 64, 3)

In [61]:
#Reading healthy images
healthy_images = []
noread = []
path_cat_images=os.path.join(path,'UnhasImagesHealthyCropped')
file_names = df_healthy.filename
row=64
col=64

for image_name in tqdm(file_names):
    try:
        image = cv2.imread(os.path.join(path_cat_images,image_name))
        image = cv2.resize(image,(row,col))
        healthy_images.append(image)
    except:
        noread.append(image_name)
        continue
        
healthy_images = np.array(healthy_images)
noread = np.array(noread)

100%|███████████████████████████████████████████████████████████████████████████████| 322/322 [00:02<00:00, 147.10it/s]


In [63]:
healthy_images.shape

(322, 64, 64, 3)